# Week8 Homework
## Working with Strings

The data frame contains notes taken by staff members. Your task is to clean it up to make it usable for analysis. 

### Tasks
* Create start date, end date columns and populate the values (If notes don't include month or day use 1 for missing value) 

i.e: 
    
    - 2018 => 2018-01-01  
    - June 2021 => 2021-06-01  
    - 2020 => 2020-01-01

* Create phone number column (If notes don't include phone numbers, use None)
* Create a column `duration` showing difference between start & end dates in years as a whole number

Note: Choose one of the following datasets to work on based on your taste! 

In [1]:
import pandas as pd
import re
from datetime import datetime

In [2]:
spicy = 'https://raw.githubusercontent.com/msaricaumbc/DS_Data/master/strings/hw_data_spicy.csv'
#less_spicy = 'https://raw.githubusercontent.com/msaricaumbc/DS_Data/master/strings/hw_data_less_spicy.csv'

In [3]:
df = pd.read_csv(spicy, index_col=0)

In [4]:
df.head()

,email,notes
0,ayalajodi@hotmail.com,start date 1982-05-20 end dt 2016
1,tmitchell@hotmail.com,P# 896.729.1768x061-STDT 2004-06-12-end date 2...
2,johnrivera@gmail.com,st date:1976/end dt 05/16/2019/pn: (895)351-43...
3,kimberly60@yahoo.com,stdt1995-03 end date December 2021 pn: 881139...
4,paulharding@jefferson-burns.com,pn: (448)006-0055x240-STDT 1983-02-ed date:08/...


In [5]:
#function to extract phone number
def phonenumber(a):
    pattern = re.compile(r'(((\+1\-)?\(?\d{3}\)?[\.\-]?\d{3}[\.\-]\d{3,4}(-\d{4})?(x\d{1,5})?)|\d{10})')
    if(pattern.search(a)):
        return pattern.search(a).group().strip()
    return 'None'

In [6]:
def dateFormatter(a):
    strp_chars = '/-:'
    a = a.strip(strp_chars)
    a = a.strip(' ')
    
    if(len(a) == 4):
        return a.strip(' ') + '-01-01'
    
    pattern1 = re.compile(r'[a-zA-Z]+')
    if('-' in a):
        dates = a.split('-')
    elif('/' in a):
        dates = a.split('/')
    elif(' ' in a):
        dates = a.split(' ')
        
    if(len(dates) == 3):
        if(len(dates[0])==4):
            year = dates[0]
            month = dates[1]
            date = dates[2]
        else:
            month = dates[0]
            date = dates[1]
            year = dates[2]
    elif(len(dates) == 2):
        date = '01'
        if(len(dates[0])==4 and dates[0].isalpha() == False):
            year = dates[0]
            month = dates[1]
        elif(len(dates[0]) == 2 and len(dates[1]) == 2):
            month = dates[0]
            if(int(dates[1]) >= 22):
                year = '19' + dates[1]
            else:
                year = '20' + dates[1]
        else:
            month = dates[0]
            year = dates[1]
    else:
        
        year = dates[0]
        month = '01'
        date = '01'
        
    if(pattern1.search(a)):
        monthnum = datetime.strptime(pattern1.search(a).group(), '%B').month
        if(monthnum > 0 and monthnum < 10):
            month = '0' + str(monthnum)
        else:
            month = str(monthnum)
    return year.strip(' ')+'-'+month.strip(' ')+'-'+date.strip(' ')

In [7]:
#function to extract startDate
def startDate(a):
    startPattern = re.compile(r'st([a-z]+)?[\s]?[a-z]+[\s\:]?',re.I)
    endPattern = re.compile(r'[\s\-\/]?e(n|d)')
    return dateFormatter(a[startPattern.search(a).end():endPattern.search(a).start()])

In [8]:
#function to extract endDate
def endDate(a):
    endDatePattern = re.compile(r'[\s\-\/]?e(n|d)(\w+)?[\s]?d([a-z]+)?[\s\:]',re.I)
    phonePattern = re.compile(r'p(n|\/)')
    if(phonePattern.search(a[endDatePattern.search(a).end():], re.I) or 'p#' in a[endDatePattern.search(a).end():].lower()):
        end = endDatePattern.search(a).end() + max(index for index, item in enumerate(a[endDatePattern.search(a).end():].lower()) if item == 'p')
    else:
        end = len(a)
    return dateFormatter(a[endDatePattern.search(a).end():end])

In [9]:
def duration(a):
    return abs(datetime.strptime(endDate(a), '%Y-%m-%d').year - datetime.strptime(startDate(a), '%Y-%m-%d').year)

In [10]:
df['phonenumber'] = [phonenumber(i) for i in df['notes']]
df['startdate'] = [startDate(i) for i in df['notes']]
df['enddate'] = [endDate(i) for i in df['notes']]
df['duration'] = [duration(i) for i in df['notes']]
df

,email,notes,phonenumber,startdate,enddate,duration
0,ayalajodi@hotmail.com,start date 1982-05-20 end dt 2016,None,1982-05-20,2016-01-01,34
1,tmitchell@hotmail.com,P# 896.729.1768x061-STDT 2004-06-12-end date 2...,896.729.1768x061,2004-06-12,2012-05-27,8
2,johnrivera@gmail.com,st date:1976/end dt 05/16/2019/pn: (895)351-43...,(895)351-4305x228,1976-01-01,2019-05-16,43
3,kimberly60@yahoo.com,stdt1995-03 end date December 2021 pn: 881139...,8811396378,1995-03-01,2021-12-01,26
4,paulharding@jefferson-burns.com,pn: (448)006-0055x240-STDT 1983-02-ed date:08/...,(448)006-0055x240,1983-02-01,2018-08-29,35
...,...,...,...,...,...,...
1995,shane14@hotmail.com,STDT 11/05 end dt 08/13,None,2005-11-01,2013-08-01,8
1996,whebert@cohen.com,STDT 2008-09/ed date:03/16,None,2008-09-01,2016-03-01,8
1997,timothyjohnson@rice.org,st date:05/87/end date 2018-07-10,None,1987-05-01,2018-07-10,31
1998,sellersjeffrey@williams.info,st date:March 2004/end dt 12/12/2016,None,2004-03-01,2016-12-12,12
